# dev data parsing

In [39]:
from lib.my_initialization import *
# from lib.searchforfile import *

In [2]:
# # import pandas as pd, os
# def parse_iolines(input_fn, printing=False):
#     n_input=-9999
#     with open(input_fn) as f:
#         trgt1='Printing Inputs...\n'
#         trgt2='Printing Outputs...\n'
#         for n,line in enumerate(f):
#             if trgt1 == line:
#                 if printing:
#                     print(f'found inputs starting after line {n}')
#                 n_input=n
#             if trgt2 == line:
#                 if printing:
#                     print(f'found outputs starting after line {n}')
#                 n_output=n
#     return n_input,n_output

In [3]:
# def parse_inputs(input_fn,n_input,printing=False):
#     if n_input==-9999:
#         if printing:
#             print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
#         return None
#     with open(input_fn) as f:
#         inputs=f.readlines()[n_input+1:n_output-1]
#     col_name_lst=[]
#     col_value_lst=[]
#     for line in inputs:
#         string=line.split(' ')[-1]
#         eid=string.find('=')
#         if eid!=-1:
#             col_name=string[:eid]
#             # try:
#             col_value=eval(string[eid+1:-1])
#             # except Exception as e:
#             #     print (string)
#             #     col_value=string[eid+1:-1]
#             col_name_lst.append(col_name)
#             col_value_lst.append(col_value)
#     return col_name_lst, col_value_lst

In [4]:
# def parse_inputs(input_fn,n_input,printing=False):
#     if n_input==-9999:
#         if printing:
#             print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
#         return None
#     with open(input_fn) as f:
#         inputs=f.readlines()[n_input+1:n_output-1]
#     col_name_lst=[]
#     col_value_lst=[]
#     for line in inputs:
#         string=line.split(' ')[-1]
#         eid=string.find('=')
#         if eid!=-1:
#             col_name=string[:eid]
#             # try:
#             col_value=eval(string[eid+1:-1])
#             # except Exception as e:
#             #     print (string)
#             #     col_value=string[eid+1:-1]
#             col_name_lst.append(col_name)
#             col_value_lst.append(col_value)
#     return col_name_lst, col_value_lst

In [5]:
# def parse_log(input_fn,qfoo,include_inputs=True,printing=False):
#     col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,printing=printing)
#     dict_inputs=dict(zip(col_name_lst,col_value_lst))
#     if not qfoo(dict_inputs):
#         return None
#     df=parse_output(input_fn, n_output)
#     #augment df with input parameters
#     if include_inputs:
#         if printing:
#             print("input parameters were:")
#             print(col_name_lst)
#             print(col_value_lst)
#             print("returning outputs as pandas.DataFrame instance")
#         for name,value in zip ( col_name_lst, col_value_lst):
#             df[name]=value
#     df.drop(columns=['dt','reflect','set_second'],inplace=True)
#     return df

In [6]:
# # Test compute_runtime_for_folder
# nb_dir='/home/timothytyree/Documents/GitHub/bgmc/python/'
# data_dir=f"{nb_dir}/data/osg_output/Log"
# runtime=compute_runtime_for_folder(folder=data_dir)
# print(f"the total recorded runtime was {runtime:.3f} years.")

# # Test parse_iolines
# data_dir=f"{nb_dir}/data/osg_output/Log"
# os.chdir(data_dir)
# input_fn_lst=os.listdir()
# input_fn=input_fn_lst[0]
# n_input,n_output=parse_iolines(input_fn, printing=False)



# input_fn_lst=get_files_in_folder(folder=data_dir,trgt='.out.')

- TODO: plot the log output over some axis of the input_parameter space  space
- TODO: measure the slope/turning points for a given log output
- TODO: learn the slope/turning points using state space
- TODO: tensorflow jupyter notebook stackoverflow 

# TODO: make a command line executable that filters Log and returns a merged pandas.Dataframe as *.csv

In [7]:
# def filter_log(input_fn,qfoo=None):
#     # qfoo_pbc=lambda dict_inputs:dict_inputs['reflect']==0 #periodic bc
#     qfoo_rbc=lambda dict_inputs:dict_inputs['reflect']==1 #reflecting bc
#     qfoo=qfoo_rbc
#     df=parse_log(input_fn,qfoo,include_inputs=True,printing=False)
#     return df

In [8]:

# TODO: for each input_fn in input_fn_lst 
# TODO:   get inputs as df_inputs
# TODO:   if input_query_function(df_inputs):
# TODO:     get outputs as df and append inputs to fields   
#return df
# TODO: return pd.concatenate(df_lst)

In [9]:
# from lib import *

In [2]:
#input qfoo
# TODO(later): prompt user for input_query_function=lambda df: foo(df)
#get folder
qfoo=eval(input('Please enter a query function of the form "lambda dict_inputs:dict_inputs["reflect"]==0": \n'))
# prompt user for save_fn
save_fn = str(input("Please enter save folder name: \n"))
nb_dir='/home/timothytyree/Documents/GitHub/bgmc/python'
data_dir=f"{nb_dir}/data/osg_output/Log"
save_folder=f"{nb_dir}/data/osg_output/{save_fn}"
try:
    os.mkdir(save_folder)
except Exception as e:
    pass
#get df
#     retval_lst=filter_folder(folder=data_dir,qfoo=None, save_folder=save_folder)
#     # df=pd.concat(df_lst).reset_index(drop=True)
#     # df.to_csv(save_dir,index=False)
#     print(f"the total number of trials matching query was {len(df_lst)}")
#     print(f"final DataFrame stored in {save_dir}")
#     beep = lambda x: os.system("echo -n '\\a';sleep 1.2;" * x)
#     beep(3)
#     print("Nota Bene: the output is currently not performing the average over logs (6x compression advisable...)")


Please enter a query function of the form "lambda dict_inputs:dict_inputs["reflect"]==0": 
lambda dict_inputs:dict_inputs["reflect"]==0
Please enter save folder name: 
filter_pbc


In [11]:
#TODO: compute average over each condition by using .mean() on the groupby of all coll_names except CollTime

In [ ]:
#TODO: make command line function save each csv to a new folder

In [3]:
folder=data_dir
input_fn_lst=get_files_in_folder(folder,trgt='.out.')
print(f"searching {len(input_fn_lst)} files...")
def routine(input_fn):
    try:
        return filter_log(input_fn,save_folder,qfoo)
    except Exception as e:
        return e
# bag = db.from_sequence(input_fn_lst, npartitions=11).map(routine)
# start = time.time()
# retval_lst = list(bag)
# print(f"the run time for filtering files was {time.time()-start:.2f} seconds.")
# return retval_lst

searching 1365397 files...


In [4]:
routine(os.path.abspath(input_fn_lst[0]))

'job.out.8883430.271624'

In [12]:
#     qfoo=eval(input('Please enter a query function of the form "lambda dict_inputs:dict_inputs["reflect"]==0":\n'))
# prompt user for save_fn
#     save_fn = str(input("Please enter save file name:\n"))
nb_dir='/home/timothytyree/Documents/GitHub/bgmc/python'
data_dir=f"{nb_dir}/data/osg_output/Log"
#     save_dir=f"{nb_dir}/data/osg_output/{save_fn}"

input_fn_lst=get_files_in_folder(folder=data_dir,trgt='.out.')
input_fn=input_fn_lst[0]
n_input,n_output=parse_iolines(input_fn, printing=False)
n_input

14

In [13]:
filter_log(input_fn,qfoo=lambda dict_inputs:dict_inputs["reflect"]==0)


,N,CollTime,r,D,L,kappa,beta,reflect,set_second,niter,dt
0,12,0.402030,0.2,4,15.8114,55,0.1,0,0,250,0.00001
1,13,0.339076,0.2,4,15.8114,55,0.1,0,0,250,0.00001
2,14,0.308924,0.2,4,15.8114,55,0.1,0,0,250,0.00001
3,15,0.270279,0.2,4,15.8114,55,0.1,0,0,250,0.00001
4,16,0.229521,0.2,4,15.8114,55,0.1,0,0,250,0.00001
...,...,...,...,...,...,...,...,...,...,...,...
684,696,0.000090,0.2,4,15.8114,55,0.1,0,0,250,0.00001
685,697,0.000090,0.2,4,15.8114,55,0.1,0,0,250,0.00001
686,698,0.000090,0.2,4,15.8114,55,0.1,0,0,250,0.00001
687,699,0.000090,0.2,4,15.8114,55,0.1,0,0,250,0.00001


In [14]:
# def parse_log(input_fn,qfoo,include_inputs=True,printing=False):
#     n_input,n_output=parse_iolines(input_fn, printing=printing)
#     if n_input==-9999:
#         if printing:
#             print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
#         return None

#     col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=printing)
#     dict_inputs=dict(zip(col_name_lst,col_value_lst))
#     if not qfoo(dict_inputs):
#         return None
#     df=parse_outputs(input_fn, n_output)
#     #augment df with input parameters
#     if include_inputs:
#         if printing:
#             print("input parameters were:")
#             print(col_name_lst)
#             print(col_value_lst)
#             print("returning outputs as pandas.DataFrame instance")
#         for name,value in zip ( col_name_lst, col_value_lst):
#             df[name]=value
#     # df.drop(columns=['dt','reflect','set_second'],inplace=True)
#     return df

In [15]:
# qfoo=lambda dict_inputs:dict_inputs["reflect"]==0
# include_inputs=True;printing=False
# n_input,n_output=parse_iolines(input_fn, printing=False)
# # if n_input==-9999:
# #     if printing:
# #         print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
# #     return None

# col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=False)
# dict_inputs=dict(zip(col_name_lst,col_value_lst))
# # if not qfoo(dict_inputs):
# #     return None
# df=parse_outputs(input_fn, n_output)
# #augment df with input parameters
# if include_inputs:
#     if printing:
#         print("input parameters were:")
#         print(col_name_lst)
#         print(col_value_lst)
#         print("returning outputs as pandas.DataFrame instance")
#     for name,value in zip ( col_name_lst, col_value_lst):
#         df[name]=value
# df.drop(columns=['dt','reflect','set_second'],inplace=True)

In [16]:
# df

In [17]:
# def parse_outputs(input_fn, n_output):
#     with open(input_fn) as f:
#         for n,line in enumerate(f):
#             if n == n_output+1:
#                 line_N = line
#             if n == n_output+2:
#                 line_CollTime =line
#     N_values=np.array([eval(s) for s in line_N[:-2].split(',')])
#     CollTime_values=np.array([eval(s) for s in line_CollTime[:-2].split(',')])
#     df=pd.DataFrame({
#         'N':N_values,
#         'CollTime':CollTime_values
#     })
#     df=df.loc[df.CollTime!=-9999]
#     df.reset_index(inplace=True,drop=True)
#     return df

# merge filtered folder results

In [34]:
# def merge_folder(folder,trgt='.out.',return_df=False):
#     '''merge all files in folder matching trgt.
#     runtime was ~10 seconds for ~3000 files.'''
#     #get all files in folder
#     assert(os.path.exists(folder))
#     input_fn_lst=get_files_in_folder(folder,trgt)
#     print(f"merging {len(input_fn_lst)} files...")
#     #merge all files in folder as df
#     df=pd.concat([pd.read_csv(fn) for fn in input_fn_lst])
#     #groupby col_names minus CollRate
#     param_names=sorted(set(df.columns.values).difference({'CollTime','Rad'}))[::-1]
#     #compute mean value
#     dfm=df.groupby(param_names).mean()
#     dfm.reset_index(inplace=True)
#     #compute CollRate and drop CollTime
#     dfm['CollRate']=1./dfm['CollTime'] #Hz
#     dfm.drop(columns=['CollTime'],inplace=True)
#     #save as csv
#     save_fn=os.path.basename(folder)+'.csv'
#     os.chdir(os.path.dirname(folder))
#     dfm.to_csv(save_fn,index=False)
#     print(f'mean Collision rates saved in\n{os.path.abspath(save_fn)}')
#     if return_df:
#         return dfm
#     else:
#         return True

In [40]:
folder=f"{nb_dir}/data/osg_output/run_5_all"
retval=merge_folder(folder, return_df=True)

merging 3242 files...
mean Collision rates saved in
/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_5_all.csv
